In [1]:
import numpy as np

In [2]:
import mne

In [3]:
import pandas as pd

In [4]:
import json

In [5]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [6]:
%load_ext autoreload

%autoreload 2
import tools

In [7]:
import numpy as np

In [8]:
import itertools

In [9]:
EMPTY_ACTION = -1

In [10]:
import itertools

In [11]:
import eeg_events

In [12]:
class OfflineJoluEngagementEnvironment():
    filename = None
    iterator = None
    event = None
    
    states = ["".join(t) for t in itertools.product(*[[str(i) for i in range(0, 6)]] * 3)]
    actions = {
        0: "o",
        1: "x",
        2: "n",
        3: "*",
        4: "u",
        5: "-",
        -1: "$"
    }
    
    def __init__(self, filename):
        self.filename = filename
    
    def reset(self):
        self.iterator = eeg_events.iterate_events(self.filename)
        self.event = next(self.iterator)
        return self.states[0]
    
    def sample(self):
        event = self.event[1]
        if not event.get("collect"):
            return self.actions[event["response"]['result']["reels"]["view"][0][0]]
        else:
            return self.actions[EMPTY_ACTION]
        
    def get_state(self):
        event_tuple = self.event
        event = event_tuple[1]
        state_array = event["response"]['result']["state"]["towerStages"]
        state = "".join([str(i) for i in state_array])
        return state
    
    def get_reward(self):
        event_tuple = self.event
        event_data = event_tuple[2]
        assert event_data.shape[0] > 0
        return np.median(event_data)
        
    def step(self, action):
        event = self.event
        
        next_state = self.get_state()
        reward = self.get_reward()
        
        self.event = next(self.iterator, None)
        
        done = not self.event
        
        return next_state, reward, done, None

In [13]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [14]:
env.reset()

Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.


'000'

In [15]:
done = False
while not done:
    print(env.sample())
    state_tuple = env.step('')
    _, _, done, _ = state_tuple
    print(state_tuple)

u
('001', 5.163911581039429, False, None)
u
('002', 3.6251907348632812, False, None)
o
('113', 3.287583887577057, False, None)
$
('000', 3.373575985431671, False, None)
-
('000', 3.4239096641540527, False, None)
-
('000', 3.104528248310089, False, None)
u
('001', 3.057699680328369, False, None)
-
('001', 3.2693721055984497, False, None)
-
('001', 3.0806952118873596, False, None)
-
('001', 3.011557877063751, False, None)
-
('001', 3.154055893421173, False, None)
-
('001', 3.3953155875205994, False, None)
*
('011', 3.498696982860565, False, None)
-
('011', 3.7934980988502502, False, None)
*
('021', 3.8866488933563232, False, None)
$
('000', 3.7224859595298767, False, None)
u
('001', 3.0336636304855347, False, None)
-
('001', 3.1165533661842346, False, None)
-
('001', 3.2759527564048767, False, None)
*
('011', 3.7076988220214844, False, None)
-
('011', 3.447341799736023, False, None)
-
('011', 2.9384170174598694, False, None)
-
('011', 3.1007161736488342, False, None)
*
('021', 3.74367421

### SARSA

In [16]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [37]:
alpha = 0.15
gamma = 0.90

In [38]:
Q = {}

In [39]:
states_number = {}

In [40]:
from tqdm import tqdm_notebook

In [41]:
for i in tqdm_notebook(range(0, 100)):
    r = 0
    state = env.reset()
    action = env.sample()
    while True:
        nextstate, reward, done, _ = env.step(action)
        if done:
            break
        nextaction = env.sample()
#         print(state, action, nextstate, nextaction)
        if (state, action) not in Q:
            Q[(state, action)] = 0
            states_number[(state, action)] = []
        if (nextstate, nextaction) not in Q:
            Q[(nextstate, nextaction)] = 0
            states_number[(nextstate, nextaction)] = []
        Q[(state, action)] += alpha * (reward + gamma * Q[(nextstate, nextaction)] - Q[(state, action)])
        states_number[(state, action)].append(reward)
        action = nextaction
        state = nextstate
        r += reward

/home/anatoli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found


Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not

In [42]:
symbols_engagement = list(filter(lambda x: x[-1] != 0, sorted(Q.items(), key=lambda x: x[1])))

In [43]:
def calculate_reward(state):
    bells = int(state[0])
    stars = int(state[1])
    horsehoes = int(state[2])
    return joker_rewards["bell"][bells] + joker_rewards["star"][stars] + joker_rewards["horsehoe"][horsehoes]

In [44]:
joker_rewards = {
    "bell": [0, 10, 40, 100, 200, 350],
    "star": [0, 3, 8, 16, 29, 50],
    "horsehoe": [0, 1, 2, 4, 7, 10],
    "biggest": 500
}

In [45]:
symbols_engagement = [(a, calculate_reward(a), len(states_number[(a, b)]), b, c) for (a, b), c in symbols_engagement]

In [46]:
sorted(symbols_engagement, key=lambda x: x[-1])

[('103', 14, 100, '$', 35.27467633943807),
 ('001', 1, 100, 'o', 37.673886718558315),
 ('002', 2, 100, 'o', 37.86267229519329),
 ('000', 0, 100, 'n', 38.10920608712064),
 ('113', 17, 100, '$', 38.45312517005525),
 ('100', 10, 100, 'u', 38.58624702881403),
 ('013', 7, 300, 'u', 38.8433375651385),
 ('012', 5, 400, '$', 38.85133285750984),
 ('013', 7, 600, '-', 38.97046260252448),
 ('000', 0, 3900, '-', 39.00036828206254),
 ('003', 4, 200, '*', 39.07178578931436),
 ('010', 3, 500, '-', 39.13364899312869),
 ('014', 10, 300, '$', 39.13635793130475),
 ('102', 12, 200, '-', 39.16529736949754),
 ('001', 1, 800, '*', 39.17969153488576),
 ('112', 15, 100, '$', 39.18696221331036),
 ('001', 1, 4000, '-', 39.192234250986324),
 ('002', 2, 200, 'n', 39.20939884655254),
 ('000', 0, 100, 'x', 39.212219234912226),
 ('000', 0, 3400, 'u', 39.21323845586168),
 ('003', 4, 300, 'u', 39.23352558348075),
 ('011', 4, 1100, '-', 39.23459734329874),
 ('004', 7, 300, '$', 39.25046576196821),
 ('003', 4, 100, 'n', 

Пример - состояние 013. Вовлеченность существенно больше, если игрок находится в состоянии 0. Что это за состояние?

Как меняются приоритеты игрока в состояниях 0 и 2

In [47]:
def increase_state(s, i):
    return s[:i] + chr(ord(s[i]) + 1) + s[i+1:]

In [48]:
def get_transition(s, a):
    if a == '-':
        return s
    if a in ['$', 'x']:
        return '000'
    if a == 'u':
        return increase_state(s, 2)
    if a == 'n':
        return increase_state(s, 0)
    if a == '*':
        return increase_state(s, 1)
    if a == 'o':
        ns = increase_state(s, 0)
        ns = increase_state(ns, 1)
        ns = increase_state(ns, 2)
        return ns

In [49]:
get_transition("000", "o")

'111'

In [50]:
rewards = {}

In [51]:
for key, value in list(Q.items()):
    if not value:
        del Q[key]

In [52]:
V = {} # V - minimum value
for (s, a), value in Q.items():
    if s not in V:
        V[s] = max(Q.values())
    if value < V[s]:
        V[s] = value

In [53]:
sorted(V.items(), key=lambda x: x[1])

[('103', 35.27467633943807),
 ('001', 37.673886718558315),
 ('002', 37.86267229519329),
 ('000', 38.10920608712064),
 ('113', 38.45312517005525),
 ('100', 38.58624702881403),
 ('013', 38.8433375651385),
 ('012', 38.85133285750984),
 ('003', 39.07178578931436),
 ('010', 39.13364899312869),
 ('014', 39.13635793130475),
 ('102', 39.16529736949754),
 ('112', 39.18696221331036),
 ('011', 39.23459734329874),
 ('004', 39.25046576196821),
 ('101', 39.4894797777533),
 ('021', 39.68255271236425),
 ('020', 39.94928353401947),
 ('022', 40.106474926785296)]

In [54]:
for (s, a), value in Q.items():
    ns = get_transition(s, a)
    if (s, a) not in rewards:
        rewards[(s, a)] = 0
    rewards[(s, a)] = Q[(s, a)] - V[ns] / gamma

In [55]:
sorted(rewards.items(), key=lambda x: x[1])

[(('103', '$'), -7.068885979584863),
 (('001', 'o'), -5.867182407342085),
 (('100', 'u'), -5.290952724245194),
 (('002', 'o'), -4.863022338201432),
 (('000', 'n'), -4.764401722672723),
 (('013', 'u'), -4.641504580755672),
 (('021', 'u'), -4.549650878251093),
 (('011', '*'), -4.453649401639474),
 (('001', '*'), -4.414305513223944),
 (('012', '*'), -4.380727771352433),
 (('003', 'u'), -4.378103040928373),
 (('011', '-'), -4.3593997048109685),
 (('102', '-'), -4.351699707721949),
 (('010', '-'), -4.34818322145874),
 (('002', 'n'), -4.307598230666947),
 (('013', '-'), -4.18880135874052),
 (('000', '*'), -4.171118504873824),
 (('010', 'u'), -4.1595818380660035),
 (('020', 'u'), -4.142441701940804),
 (('010', '*'), -4.139465163739857),
 (('003', '*'), -4.087478171950636),
 (('101', 'u'), -4.027517299466183),
 (('113', '$'), -3.8904371489676777),
 (('101', '-'), -3.7741261462397233),
 (('001', 'n'), -3.643663968809996),
 (('012', '$'), -3.492229461513091),
 (('002', 'u'), -3.3631521854711863)